In [1]:
import pandas as pd
import itertools
from constants import *
from functions import *

In [2]:
df_train = pd.read_csv("split/train.csv")

In [6]:
hedge_fund_df = {}

for ((i, xchg), (j, stk)) in itertools.product(enumerate(exchanges), enumerate(stocks)):
    df = df_train[(df_train.xchg == xchg) & (df_train.stk == stk) & (df_train.cp == "HF")]

    if stk == "A":
        df = df[df.sz != 450]
    if stk == "C":
        df = df[df.sz != 2000]

    n = len(df)
    A = np.vstack([
        df.pred_edge,
        df.px,
        df.sz,
        # df.sm,
        np.ones((1, n))]).T

    ys = (df.real_edge - df.pred_edge).values.reshape((n, -1))

    x0 = np.linalg.lstsq(A, ys, rcond=None)[0].flatten()
    hedge_fund_df[(xchg, stk)] = {
        'pred_edge_slope': x0[0],
        'px_slope': x0[1],
        'sz_slope': x0[2],
        # 'sm_slope': x0[3],
        'offset': x0[3]
    }

# hedge_fund_df.to_csv('hedge_fund_lines.csv')
hedge_fund_df = pd.DataFrame.from_dict(hedge_fund_df, orient='index')
hedge_fund_df.index = hedge_fund_df.index.rename(['xchg', 'stk'])
hedge_fund_df.to_csv("hedge_fund_lines.csv")